In [2]:
# Phase 1 — Data validation & optional cleaning for UCF101
# Paste this whole cell into train.ipynb and run.
# It will:
#  - Report class-folder counts and total videos
#  - Verify annotation files (train/test/classInd) and cross-check listed video paths
#  - Check readability of video files (first frame) for a sample or full scan and optionally move corrupt files

import os
from pathlib import Path
from collections import Counter
import json
import shutil
import cv2
from tqdm import tqdm

# -------- CONFIG (change paths if needed) --------
UCF_ROOT = Path("UCF101")   # folder with 101 class subfolders of .avi videos
ANN_DIR  = Path("UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist")  # folder with trainlist/testlist/classInd
MOVE_CORRUPT = False        # set True to move unreadable files to ./_corrupt_videos
MAX_CHECK_VIDEOS = 500      # set None to scan all videos (can be slow); useful to limit for a quick run
SAMPLE_PER_CLASS = 3        # print a few samples per class for verification
# -------------------------------------------------

def is_video_file(p: Path):
    return p.is_file() and p.suffix.lower() in {".avi", ".mp4", ".mov", ".mkv"}

def report_structure(root: Path, ann_dir: Path):
    assert root.exists(), f"UCF root not found: {root}"
    assert ann_dir.exists(), f"Annotation folder not found: {ann_dir}"
    classes = sorted([d.name for d in root.iterdir() if d.is_dir()])
    class_counts = {}
    total_videos = 0
    samples = {}
    for c in classes:
        folder = root / c
        vids = [f for f in folder.iterdir() if is_video_file(f)]
        class_counts[c] = len(vids)
        total_videos += len(vids)
        samples[c] = [str(v.name) for v in vids[:SAMPLE_PER_CLASS]]
    print(f"Found {len(classes)} class folders, total videos = {total_videos}")
    print("Top 8 classes by video count:")
    for cls, cnt in Counter(class_counts).most_common(8):
        print(f"  {cls}: {class_counts[cls]} files")
    print("\nSample files (first few per class):")
    for c in list(samples.keys())[:6]:
        print(f"  {c}: {samples[c]}")
    return classes, class_counts

def load_annotation_lists(ann_dir: Path):
    # read classInd.txt, trainlist/testlist*.txt (safe parsing)
    ann_files = list(sorted(ann_dir.glob("*.txt")))
    ann_data = {}
    for f in ann_files:
        key = f.name
        with open(f, "r", encoding="utf-8", errors="ignore") as fh:
            lines = [ln.strip() for ln in fh if ln.strip()]
        ann_data[key] = lines
    print(f"Found annotation files: {sorted(ann_data.keys())}")
    # return dictionary of filename -> lines
    return ann_data

def crosscheck_annotations(root: Path, ann_data: dict, limit_report=20):
    # Many train/test lists contain entries like: ClassName/v_ClassName_g01_c01.avi [label]
    # We'll parse and verify the existence of each referenced video path under root.
    missing = []
    total_ref = 0
    for fname, lines in ann_data.items():
        # focus on train/test lists and classInd separately
        if fname.lower().startswith("classind"):
            # classInd lines expected: "<index> <ClassName>"
            continue
        for ln in lines:
            total_ref += 1
            # split by whitespace and take first token as path
            token = ln.split()[0]
            candidate = root / token
            if not candidate.exists():
                missing.append((fname, token))
                if len(missing) <= limit_report:
                    print(f"[MISSING] {fname}: {token}")
    print(f"Checked {total_ref} annotation entries; missing references: {len(missing)}")
    if len(missing) and len(missing) > limit_report:
        print(f"...and {len(missing)-limit_report} more missing entries (truncated report).")
    return missing

def check_videos_readable(root: Path, max_videos=None, move_corrupt=False):
    moved = []
    checked = 0
    corrupt_dir = root.parent / "_corrupt_videos"
    if move_corrupt:
        corrupt_dir.mkdir(parents=True, exist_ok=True)
    for cls in sorted([d for d in root.iterdir() if d.is_dir()]):
        for vid in cls.iterdir():
            if not is_video_file(vid):
                continue
            # optional early stop
            if max_videos and checked >= max_videos:
                return {"checked": checked, "moved": moved}
            checked += 1
            try:
                cap = cv2.VideoCapture(str(vid))
                ok, frame = cap.read()
                cap.release()
                if not ok or frame is None:
                    # treat as corrupt/unreadable
                    if move_corrupt:
                        target_folder = corrupt_dir / cls.name
                        target_folder.mkdir(parents=True, exist_ok=True)
                        shutil.move(str(vid), str(target_folder / vid.name))
                        moved.append(str(vid))
                    else:
                        moved.append(str(vid))
            except Exception as e:
                moved.append(str(vid))
    print(f"Checked {checked} videos; unreadable/corrupt found: {len(moved)}")
    if move_corrupt:
        print(f"Moved corrupt files to: {corrupt_dir}")
    return {"checked": checked, "moved": moved}

# -------- Run Phase 1 checks --------
print("=== Phase1: UCF101 structure & annotation quick report ===")
classes, class_counts = report_structure(UCF_ROOT, ANN_DIR)

print("\n=== Load annotation files and cross-check listed video paths ===")
ann_data = load_annotation_lists(ANN_DIR)
missing_refs = crosscheck_annotations(UCF_ROOT, ann_data, limit_report=30)

print("\n=== Quick readability test of video files (first frame) ===")
# For speed we do a limited check by default; set MAX_CHECK_VIDEOS=None to scan all.
readability_report = check_videos_readable(UCF_ROOT, max_videos=MAX_CHECK_VIDEOS, move_corrupt=MOVE_CORRUPT)

# Save a small JSON report you can inspect later
report = {
    "classes_count": len(classes),
    "per_class_counts": class_counts,
    "annotations_files": sorted(list(ann_data.keys())),
    "missing_annotation_references_count": len(missing_refs),
    "sample_missing_refs": missing_refs[:10],
    "readability_checked": readability_report["checked"],
    "readability_unreadable_count": len(readability_report["moved"]),
    "moved_files_if_any": readability_report["moved"][:20],
}
with open("phase1_report.json", "w") as fh:
    json.dump(report, fh, indent=2)

print("\nPhase 1 report saved -> phase1_report.json")
print("Summary:", {k: report[k] for k in ["classes_count", "readability_checked", "readability_unreadable_count", "missing_annotation_references_count"]})


=== Phase1: UCF101 structure & annotation quick report ===
Found 1 class folders, total videos = 0
Top 8 classes by video count:
  UCF-101: 0 files

Sample files (first few per class):
  UCF-101: []

=== Load annotation files and cross-check listed video paths ===
Found annotation files: ['classInd.txt', 'testlist01.txt', 'testlist02.txt', 'testlist03.txt', 'trainlist01.txt', 'trainlist02.txt', 'trainlist03.txt']
[MISSING] testlist01.txt: ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi
[MISSING] testlist01.txt: ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi
[MISSING] testlist01.txt: ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi
[MISSING] testlist01.txt: ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi
[MISSING] testlist01.txt: ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi
[MISSING] testlist01.txt: ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c06.avi
[MISSING] testlist01.txt: ApplyEyeMakeup/v_ApplyEyeMakeup_g02_c01.avi
[MISSING] testlist01.txt: ApplyEyeMakeup/v_ApplyEyeMakeup_g02_c02.avi
[MISSING] testlist01.tx

In [3]:
# Fix dataset-root detection + compare classInd -> folder names
from pathlib import Path
from collections import Counter
import json

BASE = Path("UCF101")   # current folder you used
ANN_DIR = Path("UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist")
CLASSIND_PATH = ANN_DIR / "classInd.txt"

def count_video_files(p):
    exts = {".avi", ".mp4", ".mkv", ".mov"}
    files = [f for f in p.rglob("*") if f.suffix.lower() in exts]
    return len(files)

# 1) Inspect top-level children under BASE
print("BASE:", BASE.resolve())
children = sorted([c for c in BASE.iterdir()]) if BASE.exists() else []
print("Top-level children under BASE (name : type):")
for c in children[:30]:
    t = "dir" if c.is_dir() else "file"
    # if dir, count immediate subdirs & few video files if present
    subdirs = len([d for d in c.iterdir() if d.is_dir()]) if c.is_dir() else 0
    vids_immediate = len([f for f in c.iterdir() if f.is_file() and f.suffix.lower() in {'.avi','.mp4'}]) if c.is_dir() else 0
    print(f" - {c.name}  ({t})  subdirs={subdirs}  immediate_vids={vids_immediate}")

# 2) Try to auto-detect the real UCF root: a folder that contains many class subfolders
candidate = None
best_score = -1
for c in children:
    if not c.is_dir(): 
        continue
    # score by number of subdirs that themselves contain video files
    subdirs = [d for d in c.iterdir() if d.is_dir()]
    count_subdirs_with_vids = 0
    total_vids = 0
    for sd in subdirs:
        vids = [f for f in sd.iterdir() if f.is_file() and f.suffix.lower() in {'.avi','.mp4','.mkv','.mov'}]
        if vids:
            count_subdirs_with_vids += 1
            total_vids += len(vids)
    score = count_subdirs_with_vids + (total_vids/1000.0)  # heuristics
    if score > best_score:
        best_score = score
        candidate = (c, count_subdirs_with_vids, total_vids)

if candidate:
    print("\nAuto-detected candidate inner folder that looks like dataset root:")
    print(f" -> {candidate[0].as_posix()}  | classes_with_vids={candidate[1]}  | total_vids_in_these={candidate[2]}")
    SUGGESTED_ROOT = candidate[0]
else:
    # if nothing, maybe BASE itself has class folders
    print("\nNo inner candidate found; falling back to BASE as root")
    SUGGESTED_ROOT = BASE

# 3) Quick re-check of SUGGESTED_ROOT: list first 8 class folders and counts
sr = SUGGESTED_ROOT
if not sr.exists():
    raise FileNotFoundError(f"Suggested root does not exist: {sr}")
class_dirs = sorted([d for d in sr.iterdir() if d.is_dir()])
print(f"\nUsing dataset root = {sr}  | class folders found = {len(class_dirs)}")
sample = []
for d in class_dirs[:8]:
    vids = [f for f in d.iterdir() if f.is_file() and f.suffix.lower() in {'.avi','.mp4','.mkv','.mov'}]
    sample.append((d.name, len(vids)))
print("Sample class folder video counts (first 8):")
for name,cnt in sample:
    print(f" - {name}: {cnt}")

# 4) Compare classInd.txt names to folder names (exact match)
if CLASSIND_PATH.exists():
    with open(CLASSIND_PATH, "r", encoding="utf-8", errors="ignore") as fh:
        classind_lines = [ln.strip() for ln in fh if ln.strip()]
    # classInd lines typically: "<index> <ClassName>"
    class_names_from_file = [ln.split()[-1] for ln in classind_lines]
    folder_names = [d.name for d in class_dirs]
    set_file = set(class_names_from_file)
    set_folders = set(folder_names)
    missing_in_fs = sorted(list(set_file - set_folders))
    extra_folders = sorted(list(set_folders - set_file))
    print(f"\nclassInd entries found: {len(class_names_from_file)}  | folder names: {len(folder_names)}")
    print("Examples (classInd first 6):", class_names_from_file[:6])
    print("Examples (folders first 6):", folder_names[:6])
    print(f"\nclassInd names missing as folders (count={len(missing_in_fs)}): {missing_in_fs[:10]}")
    print(f"Extra folders not in classInd (count={len(extra_folders)}): {extra_folders[:10]}")
else:
    print("\nclassInd.txt not found at expected path:", CLASSIND_PATH)

# Save a small helper file so you can inspect results
out = {
    "base": str(BASE),
    "suggested_root": str(SUGGESTED_ROOT),
    "detected_classes_count": len(class_dirs),
    "sample_class_counts": sample,
    "classind_missing_count": len(missing_in_fs) if CLASSIND_PATH.exists() else None,
    "extra_folders_count": len(extra_folders) if CLASSIND_PATH.exists() else None
}
with open("phase1_root_autodetect.json","w") as fh:
    json.dump(out, fh, indent=2)

print("\nWrote helper report -> phase1_root_autodetect.json")
print("If SUGGESTED_ROOT looks correct, set UCF_ROOT = Path('<that path>') in the next cell and re-run the Phase1 checks.")


BASE: C:\Users\HP\mlresearchpaper\UCF101
Top-level children under BASE (name : type):
 - UCF-101  (dir)  subdirs=101  immediate_vids=0

Auto-detected candidate inner folder that looks like dataset root:
 -> UCF101/UCF-101  | classes_with_vids=101  | total_vids_in_these=13320

Using dataset root = UCF101\UCF-101  | class folders found = 101
Sample class folder video counts (first 8):
 - ApplyEyeMakeup: 145
 - ApplyLipstick: 114
 - Archery: 145
 - BabyCrawling: 132
 - BalanceBeam: 108
 - BandMarching: 155
 - BaseballPitch: 150
 - Basketball: 134

classInd entries found: 101  | folder names: 101
Examples (classInd first 6): ['ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'BalanceBeam', 'BandMarching']
Examples (folders first 6): ['ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'BalanceBeam', 'BandMarching']

classInd names missing as folders (count=0): []
Extra folders not in classInd (count=0): []

Wrote helper report -> phase1_root_autodetect.json
If SUGGEST

In [4]:
# Fix root and re-run Phase-1 quick checks (paste & run)
from pathlib import Path
import json, cv2, shutil
from collections import Counter
from tqdm import tqdm

# ---- set corrected root here ----
UCF_ROOT = Path("UCF101") / "UCF-101"      # <- CORRECTED root
ANN_DIR  = Path("UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist")
MOVE_CORRUPT = False
MAX_CHECK_VIDEOS = 500

def is_video_file(p): return p.is_file() and p.suffix.lower() in {'.avi','.mp4','.mkv','.mov'}

# quick counts
class_dirs = sorted([d for d in UCF_ROOT.iterdir() if d.is_dir()])
total_videos = sum(len([f for f in d.iterdir() if is_video_file(f)]) for d in class_dirs)
print(f"Using root: {UCF_ROOT.resolve()}")
print(f"Detected class folders: {len(class_dirs)}, total videos (approx): {total_videos}")
print("Sample first 8 classes and counts:")
for d in class_dirs[:8]:
    print(f" - {d.name}: {len([f for f in d.iterdir() if is_video_file(f)])} files")

# verify annotations exist & basic cross-check (small sample)
classind = ANN_DIR / "classInd.txt"
if classind.exists():
    with open(classind, 'r', errors='ignore') as fh:
        classind_names = [ln.strip().split()[-1] for ln in fh if ln.strip()]
    print(f"classInd entries: {len(classind_names)} (first 6): {classind_names[:6]}")
    print("Folder vs classInd exact-match check:", "OK" if set(classind_names)==set([d.name for d in class_dirs]) else "MISMATCH - inspect")
else:
    print("Warning: classInd.txt not found at", classind)

# lightweight readability check (first MAX_CHECK_VIDEOS videos)
checked = 0
unreadable = []
for cls in class_dirs:
    for vid in cls.iterdir():
        if not is_video_file(vid): continue
        if MAX_CHECK_VIDEOS and checked >= MAX_CHECK_VIDEOS: break
        checked += 1
        try:
            cap = cv2.VideoCapture(str(vid)); ok, fr = cap.read(); cap.release()
            if not ok or fr is None:
                unreadable.append(str(vid))
        except:
            unreadable.append(str(vid))
    if MAX_CHECK_VIDEOS and checked >= MAX_CHECK_VIDEOS: break

print(f"Readability check: checked={checked}, unreadable_found={len(unreadable)}")
out = {
    "corrected_root": str(UCF_ROOT),
    "detected_class_folders": len(class_dirs),
    "total_videos_approx": total_videos,
    "readability_checked": checked,
    "readability_unreadable_count": len(unreadable),
    "sample_unreadable": unreadable[:10]
}
with open("phase1_report_corrected.json", "w") as fh:
    json.dump(out, fh, indent=2)
print("Wrote phase1_report_corrected.json")


Using root: C:\Users\HP\mlresearchpaper\UCF101\UCF-101
Detected class folders: 101, total videos (approx): 13320
Sample first 8 classes and counts:
 - ApplyEyeMakeup: 145 files
 - ApplyLipstick: 114 files
 - Archery: 145 files
 - BabyCrawling: 132 files
 - BalanceBeam: 108 files
 - BandMarching: 155 files
 - BaseballPitch: 150 files
 - Basketball: 134 files
classInd entries: 101 (first 6): ['ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'BalanceBeam', 'BandMarching']
Folder vs classInd exact-match check: OK
Readability check: checked=500, unreadable_found=0
Wrote phase1_report_corrected.json


In [6]:
# Phase 2 — Frame extraction + optional flattened dev subset
# Paste & run this whole cell in train.ipynb
# Outputs: frames_root/<class>/<video_id>/frame_000001.jpg ... 
# Optional flattened shortcut: frames_flat/<class>/frame_<videoid>_<fnum>.jpg for quick ImageFolder use.

import os
from pathlib import Path
from tqdm import tqdm
import cv2
import shutil

# ----- CONFIG (edit as needed) -----
DATA_ROOT = Path("UCF101") / "UCF-101"          # corrected dataset root (from Phase1)
OUT_FRAMES_ROOT = Path("frames")                # where frames will be stored
FRAME_RATE = 10                                 # save every Nth frame (1 => every frame)
RESIZE = (112, 112)                             # (W,H) resize for all frames; set None to keep original
CLASSES_LIMIT = None                             # e.g. ["Basketball","Biking"] or None to process all 101
MAX_VIDEOS_PER_CLASS = 20                       # set small number for dev subset; None for all
SKIP_IF_EXISTS = True                           # skip extraction if target folder already exists
CREATE_FLAT = True                              # also create frames_flat/<class>/ for ImageFolder experiments
FLAT_PER_CLASS_LIMIT = 500                      # maximum flattened frames per class (across all videos)
# ------------------------------------

def is_video_file(p: Path):
    return p.is_file() and p.suffix.lower() in {".avi", ".mp4", ".mkv", ".mov"}

def extract_frames_from_video(video_path: Path, out_dir: Path, frame_rate:int=10, resize=None):
    out_dir.mkdir(parents=True, exist_ok=True)
    cap = cv2.VideoCapture(str(video_path))
    count = 0
    saved = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_rate == 0:
            if resize:
                frame = cv2.resize(frame, resize)
            out_file = out_dir / f"frame_{saved:06d}.jpg"
            cv2.imwrite(str(out_file), frame)
            saved += 1
        count += 1
    cap.release()
    return saved

# ---- main loop ----
DATA_ROOT = Path(DATA_ROOT)
OUT_FRAMES_ROOT = Path(OUT_FRAMES_ROOT)
OUT_FRAMES_ROOT.mkdir(parents=True, exist_ok=True)

class_dirs = sorted([d for d in DATA_ROOT.iterdir() if d.is_dir()])
if CLASSES_LIMIT:
    class_dirs = [d for d in class_dirs if d.name in CLASSES_LIMIT]

summary = {}
for cls in tqdm(class_dirs, desc="Classes"):
    vids = [v for v in cls.iterdir() if is_video_file(v)]
    vids = sorted(vids)
    if MAX_VIDEOS_PER_CLASS:
        vids = vids[:MAX_VIDEOS_PER_CLASS]
    class_out = OUT_FRAMES_ROOT / cls.name
    extracted_for_class = 0
    for vid in tqdm(vids, desc=f"  {cls.name}", leave=False):
        video_id = vid.stem
        target_dir = class_out / video_id
        if SKIP_IF_EXISTS and target_dir.exists() and any(target_dir.iterdir()):
            # assume already extracted
            extracted = len(list(target_dir.glob("*.jpg")))
            extracted_for_class += extracted
            continue
        try:
            extracted = extract_frames_from_video(vid, target_dir, frame_rate=FRAME_RATE, resize=RESIZE)
            extracted_for_class += extracted
        except Exception as e:
            print(f"Error extracting {vid}: {e}")
    summary[cls.name] = extracted_for_class

# ---- optional: create flattened ImageFolder-like structure ----
if CREATE_FLAT:
    FLAT_ROOT = Path("frames_flat")
    if FLAT_ROOT.exists():
        # do not delete by default; warn and reuse
        pass
    FLAT_ROOT.mkdir(parents=True, exist_ok=True)
    for cls, num_ex in tqdm(summary.items(), desc="Flatten classes"):
        flat_class_dir = FLAT_ROOT / cls
        flat_class_dir.mkdir(parents=True, exist_ok=True)
        # iterate through video subfolders and copy frames until limit reached
        copied = 0
        video_dirs = sorted((OUT_FRAMES_ROOT / cls).iterdir()) if (OUT_FRAMES_ROOT/cls).exists() else []
        for vd in video_dirs:
            for f in sorted(vd.glob("*.jpg")):
                if copied >= FLAT_PER_CLASS_LIMIT:
                    break
                dst = flat_class_dir / f"{vd.name}_{f.name}"
                if not dst.exists():
                    shutil.copy2(str(f), str(dst))
                    copied += 1
            if copied >= FLAT_PER_CLASS_LIMIT:
                break

# ---- summary output ----
total_frames = sum(summary.values())
total_videos = sum(1 for c in class_dirs for _ in (c.iterdir() if c.exists() else []) if is_video_file(_))
print("Frame extraction complete.")
print(f"Classes processed: {len(summary)}")
print(f"Total frames extracted (approx): {total_frames}")
print(f"Frames stored under: {OUT_FRAMES_ROOT.resolve()}")
if CREATE_FLAT:
    print(f"Flattened ImageFolder ready at: {Path('frames_flat').resolve()}")
# write tiny report
import json
with open("phase2_frames_report.json","w") as fh:
    json.dump({"classes_processed": len(summary), "per_class_frames": {k:int(v) for k,v in summary.items()}, "total_frames": int(total_frames)}, fh, indent=2)
print("Wrote phase2_frames_report.json")


Flatten classes: 100%|██████████| 101/101 [04:11<00:00,  2.50s/it]


Frame extraction complete.
Classes processed: 101
Total frames extracted (approx): 37703
Frames stored under: C:\Users\HP\mlresearchpaper\frames
Flattened ImageFolder ready at: C:\Users\HP\mlresearchpaper\frames_flat
Wrote phase2_frames_report.json


In [ ]:
# Phase 3 — Baseline CNN training (ResNet18 fine-tune)
# Paste & run in train.ipynb. Adjust CONFIG below if needed.

import os, json, time
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# --------------- CONFIG ---------------
FRAMES_FLAT = Path("frames_flat")           # imagefolder root made in Phase2
BATCH_SIZE = 32
NUM_EPOCHS = 6
LR = 1e-4
NUM_WORKERS = 4
IMG_SIZE = 224                              # resize for ResNet
CHECKPOINT_PATH = "cnn_resnet18_best.pth"
REPORT_PATH = "phase3_train_report.json"
USE_PRETRAINED = True                       # fine-tune pretrained resnet18
PIN_MEMORY = True
# --------------------------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---- transforms & dataloaders ----
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])
val_transform = transforms.Compose([
    transforms.Resize(int(IMG_SIZE*1.15)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

# split ImageFolder into train/val by folder-split (80/20)
dataset = datasets.ImageFolder(str(FRAMES_FLAT), transform=train_transform)
num_samples = len(dataset)
if num_samples == 0:
    raise RuntimeError(f"No images found in {FRAMES_FLAT}. Make sure Phase 2 created frames_flat/")

# create reproducible split
val_ratio = 0.20
num_val = int(num_samples * val_ratio)
num_train = num_samples - num_val
train_ds, val_ds = torch.utils.data.random_split(dataset, [num_train, num_val], generator=torch.Generator().manual_seed(42))

# override val dataset transform
val_ds.dataset.transform = val_transform

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)

num_classes = len(dataset.classes)
print(f"Found classes={num_classes}, total_images={num_samples}, train={num_train}, val={num_val}")

# ---- model ----
model = models.resnet18(pretrained=USE_PRETRAINED)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)
model = model.to(device)

# ---- optimizer / scheduler / loss ----
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)

# ---- training / eval functions ----
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    losses = []
    preds = []
    trues = []
    for inputs, labels in tqdm(loader, desc="Train Batches", leave=False):
        inputs = inputs.to(device); labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        preds.extend(torch.argmax(outputs.detach(), dim=1).cpu().tolist())
        trues.extend(labels.cpu().tolist())
    acc = accuracy_score(trues, preds)
    return sum(losses)/len(losses), acc

def validate(model, loader, criterion, device):
    model.eval()
    losses = []
    preds = []
    trues = []
    with torch.no_grad():
        for inputs, labels in tqdm(loader, desc="Val Batches", leave=False):
            inputs = inputs.to(device); labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            losses.append(loss.item())
            preds.extend(torch.argmax(outputs, dim=1).cpu().tolist())
            trues.extend(labels.cpu().tolist())
    acc = accuracy_score(trues, preds)
    return sum(losses)/len(losses), acc

# ---- main training loop ----
best_val_acc = 0.0
history = {"epochs": []}
start_time = time.time()

for epoch in range(1, NUM_EPOCHS+1):
    t0 = time.time()
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    scheduler.step()
    epoch_time = time.time() - t0

    print(f"Epoch {epoch}/{NUM_EPOCHS}  |  train_loss={train_loss:.4f} train_acc={train_acc:.4f}  |  val_loss={val_loss:.4f} val_acc={val_acc:.4f}  |  time={epoch_time:.1f}s")
    history["epochs"].append({
        "epoch": epoch, "train_loss": train_loss, "train_acc": train_acc,
        "val_loss": val_loss, "val_acc": val_acc, "time_s": epoch_time
    })

    # checkpoint best
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            "epoch": epoch, "model_state_dict": model.state_dict(), "optimizer_state_dict": optimizer.state_dict(),
            "val_acc": val_acc, "classes": dataset.classes
        }, CHECKPOINT_PATH)
        print(f"  -> Saved best checkpoint to {CHECKPOINT_PATH}")

total_time = time.time() - start_time
print(f"Training complete in {total_time/60:.2f} minutes. Best val_acc={best_val_acc:.4f}")

# write report
report = {"device": str(device), "num_classes": num_classes, "num_images": num_samples, "batch_size": BATCH_SIZE,
          "epochs": NUM_EPOCHS, "best_val_acc": best_val_acc, "history": history}
with open(REPORT_PATH, "w") as fh:
    json.dump(report, fh, indent=2)
print(f"Wrote training report -> {REPORT_PATH}")


Device: cpu
Found classes=101, total_images=35375, train=28300, val=7075


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\HP/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:09<00:00, 4.97MB/s]
Train Batches:   0%|          | 0/885 [00:00<?, ?it/s]c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


KeyboardInterrupt: 

In [8]:

# Quick dev-run: create small subset + freeze backbone + train few epochs (fast on CPU)
# Paste & run in train.ipynb

import shutil, random, time
from pathlib import Path
import torch, torch.nn as nn, torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# -------- CONFIG (fast) --------
SRC_FRAMES = Path("frames_flat")
DEV_ROOT   = Path("frames_dev")      # new small dev folder (ImageFolder layout)
NUM_CLASSES_DEV = 5                 # how many classes to sample for dev
IMAGES_PER_CLASS = 200              # images per class
IMG_SIZE = 112                      # smaller input -> faster
BATCH_SIZE = 16
NUM_EPOCHS = 3
LR = 1e-3
NUM_WORKERS = 2
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# -------------------------------

print("Device:", DEVICE)
# 1) Build dev subset
if DEV_ROOT.exists():
    print("frames_dev already exists — reusing it:", DEV_ROOT.resolve())
else:
    print("Creating dev subset at", DEV_ROOT)
    DEV_ROOT.mkdir(parents=True, exist_ok=True)
    classes = sorted([p.name for p in SRC_FRAMES.iterdir() if p.is_dir()])[:NUM_CLASSES_DEV]
    for cls in classes:
        dst = DEV_ROOT / cls
        dst.mkdir(parents=True, exist_ok=True)
        src_cls = SRC_FRAMES / cls
        imgs = sorted([p for p in src_cls.iterdir() if p.suffix.lower() == ".jpg"])
        if not imgs:
            # if nested video folders exist (frames out originally nested), grab from frames/<class>/*/*.jpg
            imgs = sorted([p for p in (Path("frames")/cls).rglob("*.jpg")])
        sampled = imgs[:IMAGES_PER_CLASS]
        for src in sampled:
            dst_file = dst / src.name
            if not dst_file.exists():
                shutil.copy2(src, dst_file)
    print("Dev subset created with classes:", classes)

# 2) Dataloaders
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])
val_transform = transforms.Compose([
    transforms.Resize(int(IMG_SIZE*1.15)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

dataset = datasets.ImageFolder(str(DEV_ROOT), transform=train_transform)
n = len(dataset)
if n == 0:
    raise RuntimeError("No images in dev folder. Check paths.")
val_ratio = 0.2
num_val = int(n*val_ratio)
num_train = n - num_val
train_ds, val_ds = torch.utils.data.random_split(dataset, [num_train, num_val], generator=torch.Generator().manual_seed(42))
val_ds.dataset.transform = val_transform

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

print(f"Dev images={n}, train={num_train}, val={num_val}, classes={len(dataset.classes)}")

# 3) Model (resnet18) with frozen backbone
model = models.resnet18(pretrained=True)
for p in model.parameters():
    p.requires_grad = False
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, len(dataset.classes))   # only last layer trains
model = model.to(DEVICE)

optimizer = optim.Adam(model.fc.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

# 4) Train quick
def run_epoch(train=True):
    if train:
        model.train()
        loader = train_loader
    else:
        model.eval()
        loader = val_loader
    losses=[]
    preds=[]; trues=[]
    with torch.set_grad_enabled(train):
        for X,y in loader:
            X = X.to(DEVICE); y = y.to(DEVICE)
            out = model(X)
            loss = criterion(out, y)
            if train:
                optimizer.zero_grad(); loss.backward(); optimizer.step()
            losses.append(loss.item())
            preds += torch.argmax(out.detach(), dim=1).cpu().tolist()
            trues += y.cpu().tolist()
    acc = accuracy_score(trues, preds) if trues else 0.0
    return sum(losses)/len(losses), acc

best_val = 0.0
start = time.time()
for ep in range(1, NUM_EPOCHS+1):
    t0 = time.time()
    tr_loss, tr_acc = run_epoch(train=True)
    val_loss, val_acc = run_epoch(train=False)
    took = time.time() - t0
    print(f"Epoch {ep}/{NUM_EPOCHS} | train_loss={tr_loss:.4f} train_acc={tr_acc:.4f} | val_loss={val_loss:.4f} val_acc={val_acc:.4f} | time={took:.1f}s")
    if val_acc > best_val:
        best_val = val_acc
        torch.save({"model_state": model.state_dict(), "classes": dataset.classes}, "dev_resnet18_best.pth")
        print(" -> Saved dev_resnet18_best.pth")
total = time.time()-start
print(f"Dev training complete in {total:.1f}s | best_val_acc={best_val:.4f}")

Device: cpu
Creating dev subset at frames_dev
Dev subset created with classes: ['ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'BalanceBeam']
Dev images=1000, train=800, val=200, classes=5
Epoch 1/3 | train_loss=0.8023 train_acc=0.7662 | val_loss=0.2164 val_acc=0.9950 | time=157.3s
 -> Saved dev_resnet18_best.pth
Epoch 2/3 | train_loss=0.2405 train_acc=0.9700 | val_loss=0.0998 val_acc=0.9900 | time=105.4s
Epoch 3/3 | train_loss=0.1358 train_acc=0.9812 | val_loss=0.0606 val_acc=1.0000 | time=82.8s
 -> Saved dev_resnet18_best.pth
Dev training complete in 346.8s | best_val_acc=1.0000


In [9]:
# Phase 3B — Progressive fine-tune on FULL frames_flat (paper-quality baseline, CPU-safe, GPU-ready)
# What this does (quick):
# 1) Train only the new classification head for N_FREEZE epochs.
# 2) Unfreeze layer4+fc and fine-tune for N_UNFREEZE epochs.
# Saves: resnet_progressive_best.pth and phase3_progressive_report.json

import os, json, time, math, random
from pathlib import Path
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# ============= CONFIG =============
FRAMES_FLAT = Path("frames_flat")
IMG_SIZE = 224
SEED = 42

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 8 if DEVICE.type == "cpu" else 32       # CPU-safe
NUM_WORKERS = 2 if DEVICE.type == "cpu" else 4

N_FREEZE = 3                 # head-only epochs
N_UNFREEZE = 7               # layer4+fc epochs
LR_HEAD = 1e-3
LR_FINETUNE = 1e-4
WEIGHT_DECAY = 1e-4
ACCUM_STEPS = 1 if DEVICE.type == "cpu" else 1       # set >1 if you need gradient accumulation

CHECKPOINT_BEST = "resnet_progressive_best.pth"
CHECKPOINT_INT  = "resnet_head_trained.pth"
REPORT_PATH = "phase3_progressive_report.json"
# ==================================

# Reproducibility
def set_seed(seed=SEED):
    random.seed(seed); torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = (DEVICE.type == "cuda")
set_seed(SEED)

print("Device:", DEVICE, "| batch_size:", BATCH_SIZE)

# ----- Datasets & Loaders -----
train_tf = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2,0.2,0.2,0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])
val_tf = transforms.Compose([
    transforms.Resize(int(IMG_SIZE*1.15)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

dataset = datasets.ImageFolder(str(FRAMES_FLAT), transform=train_tf)
if len(dataset) == 0:
    raise RuntimeError(f"No images found in {FRAMES_FLAT}. Run Phase 2 first.")
num_val = int(0.20 * len(dataset))
num_train = len(dataset) - num_val
train_ds, val_ds = torch.utils.data.random_split(dataset, [num_train, num_val], generator=torch.Generator().manual_seed(SEED))
val_ds.dataset.transform = val_tf

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS, pin_memory=(DEVICE.type=="cuda"))
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=(DEVICE.type=="cuda"))

num_classes = len(dataset.classes)
print(f"Images={len(dataset)} | Classes={num_classes} | Train={num_train} | Val={num_val}")

# ----- Model -----
weights = ResNet18_Weights.DEFAULT
model = resnet18(weights=weights)
in_f = model.fc.in_features
model.fc = nn.Linear(in_f, num_classes)
model = model.to(DEVICE)

# Utility: eval loop
def evaluate(model, loader, device):
    model.eval()
    crit = nn.CrossEntropyLoss()
    losses=[]; preds=[]; trues=[]
    with torch.no_grad():
        for X,y in loader:
            X,y = X.to(device), y.to(device)
            out = model(X)
            loss = crit(out,y)
            losses.append(loss.item())
            preds += torch.argmax(out, dim=1).cpu().tolist()
            trues += y.cpu().tolist()
    return (sum(losses)/len(losses) if losses else 0.0), (accuracy_score(trues,preds) if trues else 0.0)

# Training helper with (optional) grad accumulation and AMP on GPU
scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE.type=="cuda"))
def train_stage(model, params, epochs, lr, desc):
    optimizer = optim.AdamW(params, lr=lr, weight_decay=WEIGHT_DECAY)
    crit = nn.CrossEntropyLoss()
    history=[]
    global_best = {"val_acc": -1.0}
    for ep in range(1, epochs+1):
        model.train()
        losses=[]; preds=[]; trues=[]
        t0 = time.time()
        optimizer.zero_grad(set_to_none=True)
        for i,(X,y) in enumerate(tqdm(train_loader, desc=f"{desc} ep{ep}", leave=False)):
            X,y = X.to(DEVICE), y.to(DEVICE)
            with torch.cuda.amp.autocast(enabled=(DEVICE.type=="cuda")):
                out = model(X)
                loss = crit(out,y) / ACCUM_STEPS
            scaler.scale(loss).backward()
            if (i+1) % ACCUM_STEPS == 0:
                scaler.step(optimizer); scaler.update(); optimizer.zero_grad(set_to_none=True)
            losses.append(loss.item()*ACCUM_STEPS)
            preds += torch.argmax(out.detach(), dim=1).cpu().tolist()
            trues += y.cpu().tolist()
        tr_loss = sum(losses)/len(losses) if losses else 0.0
        tr_acc  = accuracy_score(trues, preds) if trues else 0.0
        val_loss, val_acc = evaluate(model, val_loader, DEVICE)
        took = time.time()-t0
        print(f"{desc} {ep}/{epochs} | tr_loss={tr_loss:.4f} tr_acc={tr_acc:.4f} | val_loss={val_loss:.4f} val_acc={val_acc:.4f} | time={took:.1f}s")
        history.append({"epoch": ep, "tr_loss": tr_loss, "tr_acc": tr_acc, "val_loss": val_loss, "val_acc": val_acc, "time_s": took})

        # checkpoint best
        if val_acc > global_best.get("val_acc", -1):
            global_best.update({"val_acc": val_acc, "epoch": ep})
            torch.save({"stage": desc, "epoch": ep, "model_state": model.state_dict(), "classes": dataset.classes}, CHECKPOINT_BEST)
            print(f" -> Saved best checkpoint: {CHECKPOINT_BEST} (val_acc={val_acc:.4f})")
    return history

# -------- Stage 1: freeze backbone, train FC --------
for p in model.parameters(): p.requires_grad = False
for p in model.fc.parameters(): p.requires_grad = True
print("Stage 1: training head only (fc) for", N_FREEZE, "epochs")
hist1 = train_stage(model, model.fc.parameters(), N_FREEZE, LR_HEAD, "Stage1")

# save intermediate
torch.save({"stage":"head_trained","model_state":model.state_dict(),"classes":dataset.classes}, CHECKPOINT_INT)
print("Saved intermediate:", CHECKPOINT_INT)

# -------- Stage 2: unfreeze layer4 + fc, fine-tune --------
for name, p in model.named_parameters():
    if name.startswith("layer4.") or name.startswith("fc."):
        p.requires_grad = True
    else:
        p.requires_grad = False

params_to_update = [p for p in model.parameters() if p.requires_grad]
print("Stage 2: fine-tuning layer4+fc for", N_UNFREEZE, "epochs")
hist2 = train_stage(model, params_to_update, N_UNFREEZE, LR_FINETUNE, "Stage2")

# Final eval + save
final_val_loss, final_val_acc = evaluate(model, val_loader, DEVICE)
torch.save({"stage":"progressive_final","model_state":model.state_dict(),"val_acc":final_val_acc,"classes":dataset.classes}, CHECKPOINT_BEST)
print(f"Saved FINAL checkpoint -> {CHECKPOINT_BEST} | final_val_acc={final_val_acc:.4f}")

# Report
report = {
    "device": str(DEVICE), "batch_size": BATCH_SIZE, "num_workers": NUM_WORKERS,
    "img_size": IMG_SIZE, "num_images": len(dataset), "num_classes": num_classes,
    "stage1_epochs": N_FREEZE, "stage2_epochs": N_UNFREEZE,
    "lr_head": LR_HEAD, "lr_finetune": LR_FINETUNE,
    "weight_decay": WEIGHT_DECAY, "accum_steps": ACCUM_STEPS,
    "history_stage1": hist1, "history_stage2": hist2,
    "final_val_acc": final_val_acc
}
with open(REPORT_PATH, "w") as f: json.dump(report, f, indent=2)
print("Wrote report:", REPORT_PATH)


Device: cpu | batch_size: 8
Images=35375 | Classes=101 | Train=28300 | Val=7075


C:\Users\HP\AppData\Local\Temp\ipykernel_7208\3152293568.py:97: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE.type=="cuda"))


Stage 1: training head only (fc) for 3 epochs


Stage1 ep1:   0%|          | 0/3538 [00:00<?, ?it/s]C:\Users\HP\AppData\Local\Temp\ipykernel_7208\3152293568.py:110: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE.type=="cuda")):


KeyboardInterrupt: 

In [ ]:
pip install pennylane pennylane-lightning torch torchvision


In [ ]:
# Phase 4 — QCNN starter (hybrid ResNet18 feature extractor + PennyLane quantum layer)
# Paste & run in train.ipynb. Defaults use frames_dev (small). Adjust CONFIG to use frames_flat if you understand cost.

import time, json, random
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PennyLane imports
import pennylane as qml
from pennylane import qnode
from pennylane import numpy as pnp
from pennylane.qnn import TorchLayer

# ----------------------- CONFIG (change as needed) -----------------------
DATA_ROOT = Path("frames_dev")      # USE frames_dev for QCNN experiments (fast). Use frames_flat only if you know the cost.
IMG_SIZE = 224
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 8 if DEVICE.type == "cpu" else 32
NUM_WORKERS = 2
SEED = 42

# Quantum circuit params
N_QUBITS = 6           # start small (4-8 qubits). More qubits => heavier simulation cost
N_Q_LAYERS = 3         # variational layers in the circuit
Q_OUTPUTS = N_QUBITS   # we will read one expectation per qubit

EPOCHS = 8
LR = 2e-4

CHECKPOINT = "qcnn_hybrid_best.pth"
REPORT = "phase4_qcnn_report.json"
# -----------------------------------------------------------------------

random.seed(SEED)
torch.manual_seed(SEED)

print("Device:", DEVICE, "| data root:", DATA_ROOT, "| batch_size:", BATCH_SIZE)
if not DATA_ROOT.exists():
    raise RuntimeError(f"Data folder not found: {DATA_ROOT} (create using Phase2/frames_dev)")

# ------------------ Data / Dataloaders ------------------
train_tf = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])
val_tf = transforms.Compose([
    transforms.Resize(int(IMG_SIZE*1.15)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

dataset = datasets.ImageFolder(str(DATA_ROOT), transform=train_tf)
if len(dataset) == 0:
    raise RuntimeError(f"No images in {DATA_ROOT}. Run Phase2 to create frames_dev or frames_flat.")

num_val = int(0.20 * len(dataset))
num_train = len(dataset) - num_val
train_ds, val_ds = torch.utils.data.random_split(dataset, [num_train, num_val], generator=torch.Generator().manual_seed(SEED))
val_ds.dataset.transform = val_tf

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=(DEVICE.type=="cuda"))
val_loader = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=(DEVICE.type=="cuda"))

num_classes = len(dataset.classes)
print(f"Images={len(dataset)} | Classes={num_classes} | Train={num_train} | Val={num_val}")

# ------------------ Feature extractor (frozen) ------------------
# Use ResNet18 up to the avgpool to get a 512-d feature vector
resnet_weights = ResNet18_Weights.DEFAULT
fe = resnet18(weights=resnet_weights)
# remove final fc: keep everything up to avgpool
modules = list(fe.children())[:-1]  # all layers except final fc
feature_extractor = nn.Sequential(*modules)  # outputs shape [B, 512, 1, 1]
for p in feature_extractor.parameters():
    p.requires_grad = False
feature_extractor = feature_extractor.to(DEVICE)

# a small wrapper to flatten feature maps to vector
class FeatureFlatten(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        # x: [B, 512, 1, 1] typical after avgpool; flatten to [B, 512]
        return torch.flatten(x, 1)

flatten = FeatureFlatten()

# ------------------ Classical reducer -> quantum input ------------------
# Reduce 512 -> N_QUBITS (scale down features before quantum encoding)
class Reducer(nn.Module):
    def __init__(self, in_features=512, out_features=N_QUBITS):
        super().__init__()
        self.fc = nn.Linear(in_features, out_features)
        self.act = nn.Tanh()  # keep values in [-1,1] useful for angle encoding
    def forward(self, x):
        # x assumed shape [B, 512] (float32)
        return self.act(self.fc(x))

reducer = Reducer().to(DEVICE)

# ------------------ Quantum circuit (PennyLane) ------------------
dev = qml.device("default.qubit", wires=N_QUBITS, shots=None)  # statevector simulator

# Define the variational circuit: angle embedding + StronglyEntanglingLayers
def qnode_circuit(inputs, weights):
    # inputs: length = N_QUBITS (torch tensor)
    # weights: shape (N_Q_LAYERS, N_QUBITS, 3) for StronglyEntanglingLayers
    qml.AngleEmbedding(inputs, wires=range(N_QUBITS), rotation='Y')  # embed via Y-rotations
    qml.templates.StronglyEntanglingLayers(weights, wires=range(N_QUBITS))
    # return expectation values for each qubit (Z)
    return [qml.expval(qml.PauliZ(w)) for w in range(N_QUBITS)]

# weight_shapes mapping required by TorchLayer
weight_shapes = {"weights": (N_Q_LAYERS, N_QUBITS, 3)}

# Create QNode (interface='torch' so it consumes/returns torch tensors)
qnode_torch = qml.QNode(qnode_circuit, dev, interface="torch", diff_method="backprop")

# Wrap as a TorchLayer
quantum_layer = TorchLayer(qnode_torch, weight_shapes).to(DEVICE)  # outputs tensor shape [B, N_QUBITS]

# ------------------ Hybrid model assembly ------------------
class HybridQCNN(nn.Module):
    def __init__(self, feature_extractor, flatten, reducer, quantum_layer, n_qubits=N_QUBITS, num_classes=num_classes):
        super().__init__()
        self.feature_extractor = feature_extractor
        self.flatten = flatten
        self.reducer = reducer
        self.quantum_layer = quantum_layer
        self.classifier = nn.Sequential(
            nn.Linear(n_qubits, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )
    def forward(self, x):
        # x: [B, C, H, W]
        feat = self.feature_extractor(x)           # [B, 512, 1, 1]
        feat = self.flatten(feat)                  # [B, 512]
        q_in = self.reducer(feat)                  # [B, N_QUBITS] values in (-1,1)
        # quantum layer expects shape [B, N_QUBITS] -> returns [B, N_QUBITS] (expectations)
        q_out = self.quantum_layer(q_in)
        out = self.classifier(q_out)
        return out

model = HybridQCNN(feature_extractor, flatten, reducer, quantum_layer, n_qubits=N_QUBITS, num_classes=num_classes).to(DEVICE)

# ------------------ Training setup ------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

def evaluate(model, loader, device):
    model.eval()
    losses = []; preds = []; trues = []
    with torch.no_grad():
        for X,y in loader:
            X,y = X.to(device), y.to(device)
            out = model(X)
            loss = criterion(out, y)
            losses.append(loss.item())
            preds += torch.argmax(out, dim=1).cpu().tolist()
            trues += y.cpu().tolist()
    return (sum(losses)/len(losses) if losses else 0.0), (accuracy_score(trues, preds) if trues else 0.0)

# ------------------ Training loop ------------------
best_val_acc = 0.0
history = {"epochs": []}
start_all = time.time()

for epoch in range(1, EPOCHS+1):
    t0 = time.time()
    model.train()
    running_losses = []
    train_preds = []; train_trues = []
    for X,y in tqdm(train_loader, desc=f"Train epoch {epoch}", leave=False):
        X,y = X.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        out = model(X)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        running_losses.append(loss.item())
        train_preds += torch.argmax(out.detach(), dim=1).cpu().tolist()
        train_trues += y.cpu().tolist()

    train_loss = sum(running_losses)/len(running_losses) if running_losses else 0.0
    train_acc = accuracy_score(train_trues, train_preds) if train_trues else 0.0
    val_loss, val_acc = evaluate(model, val_loader, DEVICE)
    epoch_time = time.time()-t0

    print(f"Epoch {epoch}/{EPOCHS} | tr_loss={train_loss:.4f} tr_acc={train_acc:.4f} | val_loss={val_loss:.4f} val_acc={val_acc:.4f} | time={epoch_time:.1f}s")
    history["epochs"].append({"epoch": epoch, "tr_loss": train_loss, "tr_acc": train_acc, "val_loss": val_loss, "val_acc": val_acc, "time_s": epoch_time})

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({"epoch": epoch, "model_state": model.state_dict(), "val_acc": val_acc, "classes": dataset.classes}, CHECKPOINT)
        print(f" -> Saved checkpoint: {CHECKPOINT}")

total_time = time.time() - start_all
print(f"QCNN training finished in {total_time/60:.2f} minutes. Best val_acc={best_val_acc:.4f}")

# ------------------ Save report ------------------
report = {
    "device": str(DEVICE), "n_qubits": N_QUBITS, "n_q_layers": N_Q_LAYERS, "epochs": EPOCHS,
    "best_val_acc": best_val_acc, "history": history, "num_images": len(dataset), "num_classes": num_classes
}
with open(REPORT, "w") as fh:
    json.dump(report, fh, indent=2)
print("Wrote report:", REPORT)


In [ ]:
# Phase 5 — QGCNN (Quantum-Gated CNN): hybrid ResNet feature extractor + quantum gate layer that modulates features
# Paste & run in train.ipynb. Recommended to run on frames_dev (small) first.

import time, json, random
from pathlib import Path
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# Pennylane imports
import pennylane as qml
from pennylane import numpy as pnp
from pennylane.qnn import TorchLayer

# ---------------- CONFIG ----------------
DATA_ROOT = Path("frames_dev")       # start with small dev; change to frames_flat later
IMG_SIZE = 224
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 8 if DEVICE.type == "cpu" else 32
NUM_WORKERS = 2
SEED = 42

# Quantum gate settings
N_QUBITS = 4         # fewer qubits => much faster; increase to experiment
N_Q_LAYERS = 2
ENCODING = "AngleEmbedding"   # embedding style
EPOCHS = 8
LR = 2e-4

CHECKPOINT = "qgcnn_hybrid_best.pth"
REPORT = "phase5_qgcnn_report.json"
# ----------------------------------------

random.seed(SEED)
torch.manual_seed(SEED)

print("Device:", DEVICE, " | Data root:", DATA_ROOT, " | batch_size:", BATCH_SIZE)

# ---- Data ----
train_tf = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
val_tf = transforms.Compose([
    transforms.Resize(int(IMG_SIZE*1.15)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

dataset = datasets.ImageFolder(str(DATA_ROOT), transform=train_tf)
if len(dataset) == 0:
    raise RuntimeError(f"No images found in {DATA_ROOT}. Create frames_dev or use frames_flat from Phase 2.")

num_val = int(0.2 * len(dataset))
num_train = len(dataset) - num_val
train_ds, val_ds = torch.utils.data.random_split(dataset, [num_train, num_val], generator=torch.Generator().manual_seed(SEED))
val_ds.dataset.transform = val_tf

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=(DEVICE.type=="cuda"))
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=(DEVICE.type=="cuda"))

num_classes = len(dataset.classes)
print(f"Images={len(dataset)} | Classes={num_classes} | Train={num_train} | Val={num_val}")

# ---- Feature extractor (frozen ResNet up to avgpool) ----
weights = ResNet18_Weights.DEFAULT
backbone = resnet18(weights=weights)
modules = list(backbone.children())[:-1]
feature_extractor = nn.Sequential(*modules)   # outputs [B,512,1,1]
for p in feature_extractor.parameters():
    p.requires_grad = False
feature_extractor = feature_extractor.to(DEVICE)

# flatten helper
class FlattenFeat(nn.Module):
    def forward(self, x):
        return torch.flatten(x, 1)
flatten = FlattenFeat()

# ---- Classical reducer: 512 -> gate_dim (equal to N_QUBITS) ----
class Reducer(nn.Module):
    def __init__(self, in_dim=512, out_dim=N_QUBITS):
        super().__init__()
        self.fc = nn.Linear(in_dim, out_dim)
        self.act = nn.Tanh()
    def forward(self, x):
        return self.act(self.fc(x))

reducer = Reducer().to(DEVICE)

# ---- Quantum gate circuit ----
dev = qml.device("default.qubit", wires=N_QUBITS, shots=None)

# circuit: embed -> variational layers -> return Z expectations
def qnode_circuit(inputs, weights):
    # inputs length = N_QUBITS
    qml.AngleEmbedding(inputs, wires=range(N_QUBITS), rotation='Y')
    qml.templates.StronglyEntanglingLayers(weights, wires=range(N_QUBITS))
    return [qml.expval(qml.PauliZ(i)) for i in range(N_QUBITS)]

weight_shapes = {"weights": (N_Q_LAYERS, N_QUBITS, 3)}
qnode = qml.QNode(qnode_circuit, dev, interface="torch", diff_method="backprop")
quantum_layer = TorchLayer(qnode, weight_shapes).to(DEVICE)   # outputs shape [B, N_QUBITS] with values in [-1,1]

# ---- Quantum-Gated Block: apply quantum gates (sigmoid -> [0,1]) to modulate classical features ----
class QuantumGatedBlock(nn.Module):
    def __init__(self, reducer, quantum_layer, in_dim=512, out_dim=128, n_qubits=N_QUBITS):
        super().__init__()
        self.reducer = reducer            # maps 512 -> n_qubits (tanh)
        self.quantum_layer = quantum_layer
        # also create a small projection of the original features down to n_qubits to multiply with gates
        self.proj = nn.Linear(in_dim, n_qubits)
        self.post = nn.Sequential(
            nn.Linear(n_qubits, out_dim),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
    def forward(self, feat):  # feat: [B,512]
        red = self.reducer(feat)             # [B, n_qubits], in (-1,1)
        q_out = self.quantum_layer(red)      # [B, n_qubits] in approximately (-1,1) (expectations)
        gates = torch.sigmoid(q_out)         # convert to [0,1] gating values
        proj_feat = self.proj(feat)          # [B, n_qubits]
        gated = proj_feat * gates            # elementwise modulation
        out = self.post(gated)               # [B, out_dim]
        return out, gates                     # return gates optionally for inspection

qg_block = QuantumGatedBlock(reducer, quantum_layer, in_dim=512, out_dim=128, n_qubits=N_QUBITS).to(DEVICE)

# ---- Full QGCNN model ----
class QGCNNHybrid(nn.Module):
    def __init__(self, feat_ext, flatten, qg_block, num_classes):
        super().__init__()
        self.feat_ext = feat_ext
        self.flatten = flatten
        self.qg_block = qg_block
        self.classifier = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )
    def forward(self, x):
        f = self.feat_ext(x)        # [B,512,1,1]
        f = self.flatten(f)         # [B,512]
        out_qg, gates = self.qg_block(f)   # out_qg [B,128]
        logits = self.classifier(out_qg)
        return logits, gates

model = QGCNNHybrid(feature_extractor, flatten, qg_block, num_classes=num_classes).to(DEVICE)

# ---- Training setup ----
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

def evaluate_model(model, loader, device):
    model.eval()
    losses=[]; preds=[]; trues=[]
    with torch.no_grad():
        for X,y in loader:
            X,y = X.to(device), y.to(device)
            out, _ = model(X)
            loss = criterion(out, y)
            losses.append(loss.item())
            preds += torch.argmax(out, dim=1).cpu().tolist()
            trues += y.cpu().tolist()
    return (sum(losses)/len(losses) if losses else 0.0), (accuracy_score(trues, preds) if trues else 0.0)

# ---- Training loop ----
best_val_acc = 0.0
history = {"epochs": []}
start_all = time.time()

for epoch in range(1, EPOCHS+1):
    t0 = time.time()
    model.train()
    running_losses=[]; tr_preds=[]; tr_trues=[]
    for X,y in tqdm(train_loader, desc=f"Train epoch {epoch}", leave=False):
        X,y = X.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        out, gates = model(X)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        running_losses.append(loss.item())
        tr_preds += torch.argmax(out.detach(), dim=1).cpu().tolist()
        tr_trues += y.cpu().tolist()
    train_loss = sum(running_losses)/len(running_losses) if running_losses else 0.0
    train_acc  = accuracy_score(tr_trues, tr_preds) if tr_trues else 0.0
    val_loss, val_acc = evaluate_model(model, val_loader, DEVICE)
    epoch_time = time.time()-t0

    print(f"Epoch {epoch}/{EPOCHS} | tr_loss={train_loss:.4f} tr_acc={train_acc:.4f} | val_loss={val_loss:.4f} val_acc={val_acc:.4f} | time={epoch_time:.1f}s")
    history["epochs"].append({"epoch":epoch,"tr_loss":train_loss,"tr_acc":train_acc,"val_loss":val_loss,"val_acc":val_acc,"time_s":epoch_time})

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({"epoch": epoch, "model_state": model.state_dict(), "val_acc": val_acc, "classes": dataset.classes}, CHECKPOINT)
        print(f" -> Saved checkpoint: {CHECKPOINT}")

total_time = time.time()-start_all
print(f"QGCNN training finished in {total_time/60:.2f} minutes. Best val_acc={best_val_acc:.4f}")

# ---- Save report ----
report = {
    "device": str(DEVICE), "n_qubits": N_QUBITS, "n_q_layers": N_Q_LAYERS, "epochs": EPOCHS,
    "best_val_acc": best_val_acc, "history": history, "num_images": len(dataset), "num_classes": num_classes
}
with open(REPORT, "w") as fh:
    json.dump(report, fh, indent=2)
print("Wrote report:", REPORT)


In [ ]:
# Phase 6 — Evaluation & Comparison script
# Paste & run in train.ipynb (after training baseline, QCNN, QGCNN).
# Outputs: comparison_report.csv, comparison_plots.png, confusion_{model}.png, and phase6_report.json

import time, json, os
from pathlib import Path
import torch, torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

# ---------------- CONFIG ----------------
FRAMES_FLAT = Path("frames_flat")   # dataset used for final evaluation (same used in training splits)
IMG_SIZE = 224
BATCH_SIZE = 16
NUM_WORKERS = 4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# checkpoint paths (change if you used different names)
CKPT_CNN  = Path("resnet_progressive_best.pth")
CKPT_QCNN = Path("qcnn_hybrid_best.pth")
CKPT_QGCNN= Path("qgcnn_hybrid_best.pth")

REPORT_CSV = "comparison_report.csv"
PLOT_PNG = "comparison_plots.png"
OUT_JSON = "phase6_report.json"
# ---------------------------------------

print("Device:", DEVICE)

# ---- Prepare dataset (use same val split method as training) ----
transform_val = transforms.Compose([
    transforms.Resize(int(IMG_SIZE*1.15)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

dataset = datasets.ImageFolder(str(FRAMES_FLAT), transform=transform_val)
if len(dataset) == 0:
    raise RuntimeError("No images found in frames_flat. Use Phase2 output.")

# create deterministic train/val split identical to training (seed 42)
val_ratio = 0.20
num_val = int(len(dataset)*val_ratio)
num_train = len(dataset) - num_val
_, val_ds = torch.utils.data.random_split(dataset, [num_train, num_val], generator=torch.Generator().manual_seed(42))
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=(DEVICE.type=="cuda"))

classes = dataset.classes
n_classes = len(classes)
print(f"Evaluation set size: {len(val_ds)} images | classes: {n_classes}")

# ---- Helpers ----
def param_count(model):
    return sum(p.numel() for p in model.parameters())

def load_cnn_model(checkpoint_path, device):
    # ResNet18 baseline architecture (same as used in training)
    model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    in_f = model.fc.in_features
    model.fc = nn.Linear(in_f, n_classes)
    model = model.to(device)
    if checkpoint_path.exists():
        ck = torch.load(checkpoint_path, map_location=device)
        # checkpoint format: (we saved dict with model_state)
        if "model_state" in ck:
            model.load_state_dict(ck["model_state"])
        elif "model_state_dict" in ck:
            model.load_state_dict(ck["model_state_dict"])
        elif "model_state_dict" not in ck and "model_state" not in ck and "epoch" in ck and "model_state_dict" in ck:
            model.load_state_dict(ck['model_state_dict'])
        else:
            # attempt to load all keys (fallback)
            try:
                model.load_state_dict(ck)
            except Exception:
                print("Warning: unable to load full checkpoint for CNN; proceeding with random init.")
    else:
        print("Warning: CNN checkpoint not found:", checkpoint_path)
    model.eval()
    return model

# QCNN and QGCNN model constructors must match your Phase4/Phase5 classes.
# We'll re-declare minimal wrappers matching their definitions used earlier so we can load saved states.

# ------- QCNN wrapper used in Phase 4 (must match) -------
import pennylane as qml
from pennylane.qnn import TorchLayer
def build_qcnn_model(n_qubits=6, n_q_layers=3):
    # feature extractor
    fe = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    modules = list(fe.children())[:-1]
    feature_extractor = nn.Sequential(*modules)
    for p in feature_extractor.parameters(): p.requires_grad = False
    flatten = nn.Flatten()
    # reducer
    class Reducer(nn.Module):
        def __init__(self, in_features=512, out_features=n_qubits):
            super().__init__()
            self.fc = nn.Linear(in_features, out_features)
            self.act = nn.Tanh()
        def forward(self,x):
            return self.act(self.fc(x))
    reducer = Reducer()
    # quantum layer (rebuild like Phase4)
    dev = qml.device("default.qubit", wires=n_qubits, shots=None)
    def qnode_circuit(inputs, weights):
        qml.AngleEmbedding(inputs, wires=range(n_qubits), rotation='Y')
        qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
        return [qml.expval(qml.PauliZ(w)) for w in range(n_qubits)]
    weight_shapes = {"weights": (n_q_layers, n_qubits, 3)}
    qnode_torch = qml.QNode(qnode_circuit, dev, interface="torch", diff_method="backprop")
    qlayer = TorchLayer(qnode_torch, weight_shapes)
    # classifier
    class HybridQCNN(nn.Module):
        def __init__(self, feat_ext, flatten, reducer, qlayer, n_qubits, n_classes):
            super().__init__()
            self.feat_ext = feat_ext
            self.flatten = flatten
            self.reducer = reducer
            self.q = qlayer
            self.classifier = nn.Sequential(nn.Linear(n_qubits,64), nn.ReLU(), nn.Dropout(0.3), nn.Linear(64, n_classes))
        def forward(self, x):
            feat = self.feat_ext(x)
            feat = self.flatten(feat)
            qin = self.reducer(feat)
            qout = self.q(qin)
            out = self.classifier(qout)
            return out
    return HybridQCNN(feature_extractor, flatten, reducer, qlayer, n_qubits, n_classes)

# ------- QGCNN wrapper used in Phase 5 (must match) -------
def build_qgcnn_model(n_qubits=4, n_q_layers=2):
    fe = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    modules = list(fe.children())[:-1]
    feature_extractor = nn.Sequential(*modules)
    for p in feature_extractor.parameters(): p.requires_grad = False
    flatten = nn.Flatten()
    class Reducer(nn.Module):
        def __init__(self, in_dim=512, out_dim=n_qubits):
            super().__init__()
            self.fc = nn.Linear(in_dim, out_dim); self.act = nn.Tanh()
        def forward(self, x): return self.act(self.fc(x))
    reducer = Reducer()
    dev = qml.device("default.qubit", wires=n_qubits, shots=None)
    def qnode_circuit(inputs, weights):
        qml.AngleEmbedding(inputs, wires=range(n_qubits), rotation='Y')
        qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
        return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]
    weight_shapes = {"weights": (n_q_layers, n_qubits, 3)}
    qnode = qml.QNode(qnode_circuit, dev, interface="torch", diff_method="backprop")
    qlayer = TorchLayer(qnode, weight_shapes)
    class QGBlock(nn.Module):
        def __init__(self, reducer, qlayer, in_dim=512, out_dim=128, n_qubits=n_qubits):
            super().__init__()
            self.reducer = reducer
            self.qlayer = qlayer
            self.proj = nn.Linear(in_dim, n_qubits)
            self.post = nn.Sequential(nn.Linear(n_qubits,out_dim), nn.ReLU(), nn.Dropout(0.3))
        def forward(self, feat):
            red = self.reducer(feat)
            q_out = self.qlayer(red)
            gates = torch.sigmoid(q_out)
            proj_feat = self.proj(feat)
            gated = proj_feat * gates
            out = self.post(gated)
            return out
    class QGCNNHybrid(nn.Module):
        def __init__(self, feat_ext, flatten, qg_block, num_classes):
            super().__init__()
            self.feat_ext = feat_ext; self.flatten = flatten; self.qg_block = qg_block
            self.classifier = nn.Sequential(nn.Linear(128,64), nn.ReLU(), nn.Dropout(0.3), nn.Linear(64,num_classes))
        def forward(self, x):
            f = self.feat_ext(x); f = self.flatten(f)
            out_qg = self.qg_block(f)
            logits = self.classifier(out_qg)
            return logits
    qg_block = QGBlock(reducer, qlayer, in_dim=512, out_dim=128, n_qubits=n_qubits)
    return QGCNNHybrid(feature_extractor, flatten, qg_block, n_classes)

# ---- Evaluate a model: compute acc, f1, confusion, inference time per sample ----
def evaluate_and_time(model, loader, device, n_warmup=10, n_samples=200):
    model = model.to(device); model.eval()
    preds=[]; trues=[]; losses=[]
    crit = nn.CrossEntropyLoss()
    # full eval metrics
    with torch.no_grad():
        for X,y in tqdm(loader, desc="Evaluation", leave=False):
            X,y = X.to(device), y.to(device)
            out = model(X)
            loss = crit(out,y)
            preds += torch.argmax(out, dim=1).cpu().tolist()
            trues += y.cpu().tolist()
            losses.append(loss.item())
    acc = accuracy_score(trues, preds)
    f1 = f1_score(trues, preds, average='macro')
    cm = confusion_matrix(trues, preds)
    # inference time per sample (use a subset for timing)
    all_inputs = []
    for X,y in loader:
        all_inputs.append(X)
        if sum([t.shape[0] for t in all_inputs]) >= n_samples:
            break
    inputs_for_timing = torch.cat(all_inputs, dim=0)[:n_samples].to(device)
    # warmup
    with torch.no_grad():
        for _ in range(n_warmup):
            _ = model(inputs_for_timing[:min(32, inputs_for_timing.shape[0])])
    # timed runs
    torch.cuda.synchronize() if device.type=="cuda" else None
    t0 = time.time()
    with torch.no_grad():
        _ = model(inputs_for_timing)
    torch.cuda.synchronize() if device.type=="cuda" else None
    total_t = time.time() - t0
    per_sample_ms = (total_t / inputs_for_timing.shape[0]) * 1000.0
    return {"acc": acc, "f1_macro": f1, "loss": float(np.mean(losses)), "cm": cm.tolist(), "latency_ms": per_sample_ms}

# ---- Load and evaluate models if checkpoints exist ----
results = []

# CNN baseline
print("\n--- Evaluating CNN baseline ---")
cnn_model = load_cnn_model(CKPT_CNN, DEVICE)
cnn_params = param_count(cnn_model)
res_cnn = evaluate_and_time(cnn_model, val_loader, DEVICE)
res_cnn.update({"model":"CNN_ResNet18", "params": cnn_params})
results.append(res_cnn)
print("CNN done:", {k:res_cnn[k] for k in ['acc','f1_macro','latency_ms','params']})

# QCNN
if CKPT_QCNN.exists():
    print("\n--- Evaluating QCNN hybrid ---")
    # use default qubit settings used for Phase4 (adjust if you changed)
    qcnn_model = build_qcnn_model(n_qubits=6, n_q_layers=3)
    qcnn_model = qcnn_model.to(DEVICE)
    ck = torch.load(CKPT_QCNN, map_location=DEVICE)
    # try load keys robustly
    ms = ck.get("model_state", ck.get("model_state_dict", ck.get("model_state_dict", None)))
    try:
        if ms is not None:
            qcnn_model.load_state_dict(ms)
        else:
            qcnn_model.load_state_dict(ck)
    except Exception as e:
        print("Warning: QCNN checkpoint load issue:", e)
    qcnn_params = param_count(qcnn_model)
    res_qcnn = evaluate_and_time(qcnn_model, val_loader, DEVICE, n_samples=100)
    res_qcnn.update({"model":"QCNN_Hybrid", "params": qcnn_params})
    results.append(res_qcnn)
    print("QCNN done:", {k:res_qcnn[k] for k in ['acc','f1_macro','latency_ms','params']})
else:
    print("QCNN checkpoint missing; skipping QCNN eval.")

# QGCNN
if CKPT_QGCNN.exists():
    print("\n--- Evaluating QGCNN hybrid ---")
    qg_model = build_qgcnn_model(n_qubits=4, n_q_layers=2)
    qg_model = qg_model.to(DEVICE)
    ck = torch.load(CKPT_QGCNN, map_location=DEVICE)
    ms = ck.get("model_state", ck.get("model_state_dict", ck))
    try:
        qg_model.load_state_dict(ms)
    except Exception as e:
        print("Warning: QGCNN checkpoint load issue:", e)
    qg_params = param_count(qg_model)
    res_qg = evaluate_and_time(qg_model, val_loader, DEVICE, n_samples=100)
    res_qg.update({"model":"QGCNN_Hybrid", "params": qg_params})
    results.append(res_qg)
    print("QGCNN done:", {k:res_qg[k] for k in ['acc','f1_macro','latency_ms','params']})
else:
    print("QGCNN checkpoint missing; skipping QGCNN eval.")

# ---- Save numeric report ----
df = pd.DataFrame(results)
df = df[['model','params','acc','f1_macro','loss','latency_ms','cm']]
df.to_csv(REPORT_CSV, index=False)
print("\nSaved CSV report ->", REPORT_CSV)

# ---- Plot comparison bar chart (accuracy, latency, params) ----
plt.figure(figsize=(10,5))
x = df['model']
accs = df['acc']
lat = df['latency_ms']
params_m = df['params'] / 1e6

ax1 = plt.subplot(1,3,1)
ax1.bar(x, accs); ax1.set_title("Validation Accuracy"); ax1.set_ylim(0,1)
ax2 = plt.subplot(1,3,2)
ax2.bar(x, lat); ax2.set_title("Inference latency (ms/sample)")
ax3 = plt.subplot(1,3,3)
ax3.bar(x, params_m); ax3.set_title("Parameter count (M)")
ax3.set_ylabel("Millions")

plt.tight_layout()
plt.savefig(PLOT_PNG, dpi=200)
print("Saved comparison plot ->", PLOT_PNG)

# ---- Save confusion matrices and JSON ----
out_json = {"device": str(DEVICE), "results": []}
for r in results:
    model_name = r.get("model")
    cm = np.array(r.get("cm"))
    # save cm figure
    fig, ax = plt.subplots(figsize=(6,5))
    im = ax.imshow(cm, cmap="Blues")
    ax.set_title(f"Confusion matrix: {model_name}")
    ax.set_xlabel("Predicted"); ax.set_ylabel("True")
    plt.colorbar(im, ax=ax)
    plt.savefig(f"confusion_{model_name}.png", dpi=150)
    plt.close(fig)
    # append textual results
    out_json["results"].append({k: (v if not hasattr(v, "tolist") else v) for k,v in r.items() if k!='cm'})
    out_json["results"][-1]["cm_shape"] = cm.shape

with open(OUT_JSON, "w") as fh:
    json.dump(out_json, fh, indent=2)
print("Saved JSON report ->", OUT_JSON)

print("\nPhase 6 complete. Files generated:", REPORT_CSV, PLOT_PNG, OUT_JSON)
